In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
from functools import reduce


In [33]:
import sys
sys.path.append("..") 
from models import mathsModels as mvm


In [34]:
def download_sp500_d():   
    symbol = "^GSPC" 
    start_date = "2000-01-01"
    end_date = None
    output_file = "../data/sp500_daily_data.csv"

    data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
    data.to_csv(output_file)

In [35]:
def download_sp500_m():   
    symbol = "^GSPC" 
    interval = "1m"
    period = "7d"
    output_file = "../data/sp500_intraday_1m.csv"

    data = yf.download(symbol, interval=interval, period=period)
    data.to_csv(output_file)

In [36]:
def load_csv(file_path, date_column, rename_columns):
    df = pd.read_csv(file_path)
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.rename(columns=rename_columns)
    return df

In [37]:
def load_data():
    global inflation, cpi_df, daily_volume_df, spx_volume_df, treasury_yield_df, sp500_daily_df, sp500_intraday_df, gdp, mortage_rate, unemployement, fed_fund_rate
    inflation = load_csv("../../data/Inflation.csv", "observation_date", {"observation_date": "Date", "T10YIE": "Inflation"})
    cpi_df = load_csv("../../data/CPI.csv", "observation_date", {"observation_date": "Date", "CPIAUCSL": "CPI"})
    daily_volume_df = load_csv("../../data/daily_volume_2024_1.csv", "Trade Date", {"Trade Date": "Date", "Volume": "Daily_Volume"})
    spx_volume_df = load_csv("../../data/daily_volume_SPX_2020-01-01_2024-12-18.csv", "Trade Date", {"Trade Date": "Date", "Volume": "SPX_Volume"})
    treasury_yield_df = load_csv("../../data/Market Yield.csv", "observation_date", {"observation_date": "Date", "DGS10": "Treasury_Yield"})
    sp500_daily_df = load_csv("../../data/sp500_daily.csv", "Date", {"Adj Close": "SP500_Adj_Close"})
    gdp=load_csv("../../data/GDP.csv", "observation_date", {"observation_date": "Date"})
    unemployement=load_csv("../../data/UNRATE.csv", "observation_date", {"observation_date": "Date", "UNRATE":"unemployement"})
    mortage_rate=load_csv("../../data/MORTGAGE30US.csv", "observation_date", {"observation_date": "Date", "MORTGAGE":"mortage"})
    fed_fund_rate=load_csv("../../data/Federal_Funds_Effective_Rate.csv", "observation_date", {"observation_date": "Date", "FEDFUNDS":"fed_fund_rate"})

    #sp500_intraday_df = load_csv("../../data/sp500_minute.csv", "Datetime", {"Adj Close": "SP500_Intraday_Adj_Close"})

In [38]:
def data_cleaning():
    global sp500_intraday_agg, sp500_daily_df, daily_volume_df, spx_volume_df, gdp, inflation, cpi_df, treasury_yield_df, mortage_rate, unemployement, fed_fund_rate
    #sp500_intraday_df["Date"] = sp500_intraday_df["Datetime"].dt.date
    #sp500_intraday_agg = sp500_intraday_df.groupby("Date")["SP500_Intraday_Adj_Close"].mean().reset_index()
    #sp500_intraday_agg["Date"] = pd.to_datetime(sp500_intraday_agg["Date"])

    sp500_daily_df["Date"] = pd.to_datetime(sp500_daily_df["Date"])
    daily_volume_df["Date"] = pd.to_datetime(daily_volume_df["Date"])
    spx_volume_df["Date"] = pd.to_datetime(spx_volume_df["Date"])
    gdp["Date"] = pd.to_datetime(gdp["Date"])
    inflation["Date"] = pd.to_datetime(inflation["Date"])
    cpi_df["Date"] = pd.to_datetime(cpi_df["Date"])
    treasury_yield_df["Date"] = pd.to_datetime(treasury_yield_df["Date"])
    unemployement["Date"] = pd.to_datetime(unemployement["Date"])
    mortage_rate["Date"] = pd.to_datetime(mortage_rate["Date"])
    fed_fund_rate["Date"] = pd.to_datetime(fed_fund_rate["Date"])



In [39]:
def data_filling(df, date_column, value_column):
    df = df.set_index(date_column)
    df = df.asfreq('D')
    df[value_column] = df[value_column].fillna(method='ffill')
    return df.reset_index()

In [40]:
def fill():
    global cpi_df, gdp, mortage_rate, unemployement, fed_fund_rate
    cpi_df=data_filling(cpi_df, "Date", "CPI")
    gdp=data_filling(gdp, "Date", "GDP")
    fed_fund_rate=data_filling(fed_fund_rate, "Date", "fed_fund_rate")
    unemployement=data_filling(unemployement, "Date", "unemployement")
    mortage_rate=data_filling(mortage_rate, "Date", "mortage")

    

In [41]:
def data_merging():
    data_cleaning()
    #all_dfs = [breakeven_df, cpi_df, treasury_yield_df, sp500_daily_df, sp500_intraday_agg, daily_volume_df, spx_volume_df]
    #all_dfs = [breakeven_df, cpi_df, treasury_yield_df, sp500_daily_df, daily_volume_df, spx_volume_df]
    all_dfs = [inflation, cpi_df, treasury_yield_df, sp500_daily_df, gdp, mortage_rate, unemployement, fed_fund_rate]
    merged_df = reduce(lambda left, right: pd.merge(left, right, on="Date", how="inner"), all_dfs)

    global cleaned_df
    cleaned_df = merged_df.dropna()

In [42]:
def rename_fields():
    global cleaned_df
    cleaned_df = cleaned_df.rename(columns={"10Y_Breakeven_Inflation_Rate": "Inflation"})
    cleaned_df = cleaned_df.rename(columns={"10Y_Treasury_Yield": "Market_Yield"})
    #cleaned_df = cleaned_df.rename(columns={"Ancien_Nom": "Nouveau_Nom"})


In [43]:
def add_volatility():
    global cleaned_df
    cleaned_df['volatility'] = (cleaned_df['High'] - cleaned_df['Low']) / cleaned_df['Low'] * 100
    cleaned_df['volatility_forcast'] = cleaned_df['volatility'].shift(-1)
    # plus returns
    #cleaned_df['returns'] = np.log(cleaned_df['Close'] / cleaned_df['Close'].shift(1))
    cleaned_df['returns'] = cleaned_df['Close'] - cleaned_df['Close'].shift(1)




In [44]:
def add_models_data():
    lambda_ = 0.94
    cleaned_df['EWMA_VM'] = mvm.EWMA_VM(cleaned_df['returns'].fillna(0).values, lambda_)

    omega, alpha, beta = 0.0001, 0.05, 0.8
    cleaned_df['GARCH_VM'] = mvm.GARCH_VM(cleaned_df['returns'].fillna(0).values, omega, alpha, beta)

    omega, alpha, beta, gamma = 0.0001, 0.05, 0.9, -0.1
    cleaned_df['EGARCH_VM'] = mvm.EGARCH_VM(cleaned_df['returns'].fillna(0).values, omega, alpha, beta, gamma)

    cleaned_df['RogersSatchell_VM'] = mvm.RogersSatchell_VM(cleaned_df['High'].values,cleaned_df['Low'].values,cleaned_df['Open'].values,cleaned_df['Close'].values)


    cleaned_df['garman_klass'] = mvm.GarmanKlass_VM(
    cleaned_df['Open'].values,
    cleaned_df['Close'].values,
    cleaned_df['High'].values,
    cleaned_df['Low'].values
    )

    cleaned_df['parkinson'] = mvm.parkinson_VM(
        cleaned_df['High'].values,
        cleaned_df['Low'].values
    )

    cleaned_df['yang_zhang'] = mvm.Yang_Zhang_VM(
        cleaned_df['Open'].values,
        cleaned_df['Close'].values,
        cleaned_df['High'].values,
        cleaned_df['Low'].values
    )

    
   

    # Vectorized Black-Scholes call price
    cleaned_df['BS_30d_ATM_Call'] = mvm.blackScholes_VM(
        S = cleaned_df['Close'].values,
        K = cleaned_df['Close'].values,
        T = 30 / 252,
        r = cleaned_df['Treasury_Yield'].values / 100, # percent to decimal,
        sigma = cleaned_df['volatility'].values,
        option_type = "C"
    )



    
    # ARCH volatility
    lags = 5
    arch_volatility = mvm.ARCH_VM(cleaned_df['returns'].values, lags=lags)
    #cleaned_df['arch_volatility'] = np.concatenate((np.full(lags, np.nan), arch_volatility))



In [45]:
def norm():
    global cleaned_df, df_normalized

    date_column = 'Date'

    numeric_columns = cleaned_df.drop(columns=[date_column])

    normalized_numeric_columns = (numeric_columns - numeric_columns.min()) / (numeric_columns.max() - numeric_columns.min())

    df_normalized = cleaned_df[[date_column]].join(normalized_numeric_columns)


In [46]:
def dataPreprocessing():
    global cleaned_df, df_normalized
    load_data()
    data_cleaning()
    fill()
    data_merging()
    add_volatility()
    add_models_data()
    norm()
    #df_normalized['move'] = (df_normalized['Close'] > df_normalized['Close'].shift(1)).astype(int)
    df_normalized['move'] = (df_normalized['Close'].shift(-1) > df_normalized['Close']).astype(int)

    df_normalized=df_normalized.dropna()
    df_normalized = df_normalized.round(4)
    df_normalized.to_csv("../../data/dataV.csv", index=False)
    #cleaned_df=cleaned_df.dropna()
    #cleaned_df = cleaned_df.round(4)
    #cleaned_df.to_csv("../../data/dataV.csv", index=False)


In [47]:
dataPreprocessing()

In [48]:
df_normalized


,Date,Inflation,CPI,Treasury_Yield,Open,High,Low,Close,SP500_Adj_Close,Volume,...,returns,EWMA_VM,GARCH_VM,EGARCH_VM,RogersSatchell_VM,garman_klass,parkinson,yang_zhang,BS_30d_ATM_Call,move
1,2003-01-03,0.5302,0.0000,0.7447,0.0477,0.0447,0.0492,0.0482,0.0482,0.0698,...,0.5843,0.1470,0.0472,0.6970,0.0858,0.0718,0.0703,0.0546,0.0288,1
2,2003-01-06,0.5336,0.0000,0.7532,0.0476,0.0490,0.0503,0.0525,0.0525,0.0973,...,0.6219,0.1413,0.0377,0.5021,0.1080,0.1360,0.2206,0.1746,0.1102,0
3,2003-01-07,0.5302,0.0000,0.7426,0.0518,0.0488,0.0527,0.0512,0.0512,0.1071,...,0.5742,0.1395,0.0319,0.3563,0.0869,0.0833,0.0957,0.0749,0.0434,0
4,2003-01-08,0.5604,0.0000,0.7342,0.0505,0.0472,0.0503,0.0485,0.0485,0.1001,...,0.5617,0.1343,0.0256,0.2846,0.0664,0.0806,0.1348,0.1061,0.0633,1
5,2003-01-09,0.5839,0.0000,0.7743,0.0478,0.0483,0.0506,0.0522,0.0522,0.1085,...,0.6169,0.1306,0.0212,0.2518,0.0499,0.0878,0.1723,0.1360,0.0849,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5404,2024-06-24,0.7315,0.9963,0.7869,0.9916,0.9932,0.9951,0.9919,0.9919,0.3010,...,0.5549,0.1704,0.0103,0.0000,0.0834,0.0591,0.0597,0.0465,0.2251,1
5405,2024-06-25,0.7315,0.9963,0.7827,0.9919,0.9895,0.9948,0.9963,0.9963,0.2915,...,0.6237,0.1661,0.0094,0.0000,0.0438,0.0310,0.0313,0.0275,0.1339,1
5406,2024-06-26,0.7450,0.9963,0.8017,0.9919,0.9916,0.9960,0.9981,0.9981,0.2890,...,0.6006,0.1635,0.0094,0.0000,0.0426,0.0346,0.0396,0.0317,0.1614,1
5407,2024-06-27,0.7450,0.9963,0.7954,0.9945,0.9932,0.9992,0.9991,0.9991,0.2913,...,0.5941,0.1578,0.0078,0.0000,0.0345,0.0250,0.0259,0.0197,0.1170,0


In [49]:
def smaller_df():
    df_s = df_normalized[['Date', 'Open', 'High', 'Low', 'Close', 'SP500_Adj_Close','move', 'Volume', 'volatility', 'volatility_forcast', 'returns', 'GARCH_VM']]

    df_s['High']=abs(df_s['Open'] - df_s['High'])
    df_s['Low']=abs(df_s['Open'] - df_s['Low'])
    df_s['Close']=abs(df_s['Open'] - df_s['Close'])


    df_s.to_csv("../../data/dataS.csv", index=False)

smaller_df()


C:\Users\paula\AppData\Local\Temp\ipykernel_16036\3605383329.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['High']=abs(df_s['Open'] - df_s['High'])
C:\Users\paula\AppData\Local\Temp\ipykernel_16036\3605383329.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Low']=abs(df_s['Open'] - df_s['Low'])
C:\Users\paula\AppData\Local\Temp\ipykernel_16036\3605383329.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe